<a href="https://colab.research.google.com/github/KrishPraba/Pump-it-Up-Data-Mining-the-Water-Table/blob/main/Pump_it_Up_Modelling_%26_Post_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#**Installing Necessary Libraries**

In [3]:
!pip install catboost
!pip install ipywidgets
!pip install shap
!jupyter nbextension enable --py widgetsnbextension

     |████████████████████████████████| 67.4 MB 25 kB/s 
     |████████████████████████████████| 356 kB 4.1 MB/s 
  Created wheel for shap: filename=shap-0.39.0-cp37-cp37m-linux_x86_64.whl size=491644 sha256=8083c30c86900181c1d4402d0061ed1199df918c61447a6b19e5c4009451edb2
  Stored in directory: /root/.cache/pip/wheels/ca/25/8f/6ae5df62c32651cd719e972e738a8aaa4a87414c4d2b14c9c0
Successfully built shap
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [4]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, precision_recall_fscore_support, confusion_matrix
import re
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from catboost import CatBoostClassifier, Pool, metrics, cv
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.multioutput import MultiOutputClassifier

from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

#**Loading Data**

In [10]:
import pandas as pd

train_test_unencoded = pd.read_csv( "/content/drive/MyDrive/Machine Learning/preprocessed_unencoded_data.csv" )   # Unencoded features dataset used for CATboost

train_test_encoded = pd.read_csv( "/content/drive/MyDrive/Machine Learning/preprocessed_encoded_data.csv" )       # Encoded features dataset used for other classifiers

labels_df = pd.read_csv( "/content/drive/MyDrive/Machine Learning/Train_set_labels.csv" )                         # Labels set for the training set

In [13]:
# Seperating train and test set of both encoded and unencoded dataset

test_df_unencoded = train_test_unencoded.loc[train_test_unencoded['from'] == 'test']
test_df_unencoded.drop(columns=['from','Unnamed: 0'],inplace=True)

train_values_df_unencoded = train_test_unencoded.loc[train_test_unencoded['from'] == 'train']
train_values_df_unencoded.drop(columns=['from','Unnamed: 0'],inplace=True)

test_df_encoded = train_test_encoded.loc[train_test_encoded['from'] == 'test']
test_df_encoded.drop(columns=['from','Unnamed: 0'],inplace=True)

train_values_df_encoded = train_test_encoded.loc[train_test_encoded['from'] == 'train']
train_values_df_encoded.drop(columns=['from','Unnamed: 0'],inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [14]:
print(train_values_df_encoded.shape)
print(test_df_encoded.shape)
print(labels_df.shape)
print(train_values_df_unencoded.shape)
print(test_df_unencoded.shape)

(59400, 28)
(14850, 28)
(59400, 2)
(59400, 28)
(14850, 28)


In [ ]:
train_values_df_encoded.info()

In [15]:
labels_df = labels_df.drop(columns=['id'])

In [17]:
X_unencoded =train_values_df_unencoded
Y =labels_df
X_encoded =train_values_df_encoded


#**Post-processing**

**Feature Importance**

---



**Random Forest**

In [ ]:
clf=RandomForestClassifier(n_estimators=100,max_features='auto',random_state=0,max_depth=14,class_weight='balanced')
rfecv = RFECV(estimator=clf, step=1, cv=StratifiedKFold(),scoring='accuracy')
rfecv.fit(X_encoded, Y)

print("Optimal number of features : %d" % rfecv.n_features_)

plt.figure(figsize=(5,5))
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

In [ ]:
print('\033[1m' + "Feature grid scores" + '\033[0m')
print(rfecv.grid_scores_)

print('\033[1m' + "Feature support" + '\033[0m')
print(rfecv.support_)

print('\033[1m' + "Feature rank" + '\033[0m')
print(rfecv.ranking_)

array([0.56926599, 0.61791375, 0.64288162, 0.67131855, 0.68062233,
       0.69063577, 0.69878305, 0.70530112, 0.70532739])

In [ ]:
rfecv_features = list(X.columns.values[rfecv.support_])
print(rfecv_features)

**SHAP**

In [ ]:
output = shap.TreeExplainer(rfecv)
shap_values = output.shap_values(X_valid)
shap.summary_plot(shap_values, X_valid)

In [ ]:
shap.summary_plot(shap_values[0], X_valid)      #Functional

In [ ]:
shap.summary_plot(shap_values[1], X_valid)      #Functional but needs repair

In [ ]:
shap.summary_plot(shap_values[2], X_valid)      #Non Functional

#**Building CAT Boost Model**

In [18]:
categorical_features_indices = np.where(X_unencoded.dtypes != np.float)[0]

In [19]:
categorical_features_indices

array([ 0,  2,  3,  4,  5,  8,  9, 10, 11, 12, 13, 14, 15, 17, 18, 19, 21,
       22, 23, 24, 25, 26, 27])

In [ ]:
X_train, X_validation, y_train, y_validation = train_test_split(X_unencoded, Y, train_size=0.75, random_state=42)

In [27]:
model = CatBoostClassifier(
    loss_function = 'MultiClass'
)

In [ ]:
model.fit(
    X_train, y_train,
    cat_features=categorical_features_indices,
    verbose = False
);

In [ ]:
print('Simple model validation accuracy: {:.4}'.format(
    accuracy_score(y_validation, model.predict(X_validation))
))

Simple model validation accuracy: 0.8122


In [29]:
model.fit(
    X_unencoded, Y,
    cat_features=categorical_features_indices,
    # verbose = False
);

Learning rate set to 0.097451
0:	learn: 1.0203849	total: 1.22s	remaining: 20m 14s
1:	learn: 0.9570950	total: 2.38s	remaining: 19m 49s
2:	learn: 0.9058445	total: 3.29s	remaining: 18m 14s
3:	learn: 0.8633110	total: 4.39s	remaining: 18m 14s
4:	learn: 0.8277050	total: 5.33s	remaining: 17m 41s
5:	learn: 0.7960695	total: 6.29s	remaining: 17m 21s
6:	learn: 0.7702599	total: 7.28s	remaining: 17m 13s
7:	learn: 0.7478506	total: 8.23s	remaining: 17m 1s
8:	learn: 0.7284047	total: 9.13s	remaining: 16m 45s
9:	learn: 0.7108674	total: 10.2s	remaining: 16m 47s
10:	learn: 0.6945305	total: 11.3s	remaining: 16m 53s
11:	learn: 0.6811261	total: 12.2s	remaining: 16m 47s
12:	learn: 0.6689795	total: 13.2s	remaining: 16m 44s
13:	learn: 0.6578610	total: 14.3s	remaining: 16m 47s
14:	learn: 0.6481202	total: 15.4s	remaining: 16m 50s
15:	learn: 0.6389919	total: 16.3s	remaining: 16m 43s
16:	learn: 0.6300570	total: 17.3s	remaining: 16m 39s
17:	learn: 0.6219806	total: 18.2s	remaining: 16m 31s
18:	learn: 0.6150145	total:

# **Other Modelling Approaches**

In [20]:
#Function to compute accuracy, f1 score, precision, recall and confusion matrix

def compute_metrics(pred,ground_labels):
    labels_all = ground_labels
    preds_all = list(pred)
    
    precision, recall, f1, _ = precision_recall_fscore_support(labels_all, preds_all)
    acc = accuracy_score(labels_all, preds_all)
    confusion_mat = confusion_matrix(labels_all, preds_all)
    # tn, fp, fn, tp = confusiton_mat.ravel()
    out_dict = {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'confusiton_mat': confusion_mat
    }
    return out_dict

In [21]:
X_train, X_val, y_train, y_val = train_test_split(X_encoded, Y, test_size=0.3, random_state=142,stratify=Y)

**Random Forest Classifier**

---



In [22]:
clf=RandomForestClassifier(n_estimators=100,max_features='auto',random_state=0,max_depth=14)
rfecv = RFECV(estimator=clf, step=1, cv=StratifiedKFold(),scoring='accuracy')
rfecv.fit(X_train, y_train.ravel())

predicted_y = rfecv.predict(X_val)
result = compute_metrics(predicted_y,y_val)

print(result['accuracy'])

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.7919191919191919


**Ada Boost Classifier**

---



In [23]:
ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=14), n_estimators=100,algorithm="SAMME.R", learning_rate=0.0001)
ada_clf.fit(X_train,y_train.ravel())

predicted_y = ada_clf.predict(X_val)
result = compute_metrics(predicted_y,y_val)

print(result['accuracy'])

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.7605499438832772


**Other Classifiers used**

---



In [24]:
classifiers=[['Decision Tree Classification :',DecisionTreeClassifier()],
       ['Gradient Boosting Classification :', GradientBoostingClassifier()],
       ['Extra Tree Classification :', ExtraTreesClassifier()],
       ['Support Vector Classification :',SVC()]]
cla_pred=[]

In [25]:
for name,model1 in classifiers:
    print("-----------"+name+"-------------")
    model1.fit(X_train,y_train.values.ravel())
    predicted_y = model1.predict(X_val)
    score=compute_metrics(predicted_y,y_val)
    cla_pred.append(score)
    print()

-----------Logistic Regression :-------------


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



-----------Decision Tree Classification :-------------

-----------Gradient Boosting Classification :-------------

-----------Extra Tree Classification :-------------

-----------K-Neighbors Classification :-------------

-----------Support Vector Classification :-------------


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



-----------Gaussian Naive Bayes :-------------



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
for i in range(len(classifiers)):
  # print("-----------"+classifiers[i][0]+"-------------")
  result = cla_pred[i]
  print('\033[1m' + classifiers[i][0] + '\033[0m')
  print(result['accuracy'])
  print()

Logistic Regression :
0.5386083052749719

Decision Tree Classification :
0.7271043771043771

Gradient Boosting Classification :
0.7508417508417509

Extra Tree Classification :
0.8038720538720538

K-Neighbors Classification :
0.49854096520763186

Support Vector Classification :
0.5430976430976431

Gaussian Naive Bayes :
0.6222783389450056



#**Submission**

In [30]:
test_df_unencoded.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14850 entries, 59400 to 74249
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 14850 non-null  int64  
 1   amount_tsh         14850 non-null  float64
 2   date_recorded      14850 non-null  object 
 3   funder             14850 non-null  object 
 4   gps_height         14850 non-null  int64  
 5   installer          14850 non-null  object 
 6   longitude          14850 non-null  float64
 7   latitude           14850 non-null  float64
 8   wpt_name           14850 non-null  object 
 9   basin              14850 non-null  object 
 10  subvillage         14850 non-null  object 
 11  region             14850 non-null  object 
 12  region_code        14850 non-null  int64  
 13  district_code      14850 non-null  int64  
 14  lga                14850 non-null  object 
 15  ward               14850 non-null  object 
 16  population        

In [ ]:
y_pred = model1.predict(test_df_encoded)     #if other classifiers are used

In [31]:
y_pred = model.predict(test_df_unencoded)   #if CAT BOOST is used

In [32]:
prediction = pd.DataFrame(
    y_pred
)

In [33]:
submission_df = pd.read_csv("/content/drive/MyDrive/Machine Learning/SubmissionFormat.csv")
print(submission_df.shape)

(14850, 2)


In [34]:
submission_df['status_group'] = prediction[0]
submission_df.info

<bound method DataFrame.info of           id    status_group
0      50785      functional
1      51630      functional
2      17168      functional
3      45559  non functional
4      49871      functional
...      ...             ...
14845  39307  non functional
14846  18990      functional
14847  28749      functional
14848  33492      functional
14849  68707  non functional

[14850 rows x 2 columns]>


In [36]:
submission_df.to_csv('/content/drive/MyDrive/Machine Learning/my_submission15.csv')